# Data Science

Debido al requerimiento del cliente, se procederan a reordenar la data, para obtener un set de datos horarios donde cada fila represente la cantidad de viajes realizados en esa hora en especifico(hora de comienzo del viaje).

In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet('Data\process_taxis.parquet',engine='pyarrow')

In [5]:
df.set_index(keys='pickup_datetime',inplace=True)

In [7]:
df.index

DatetimeIndex(['2023-01-01 00:00:00', '2023-01-01 00:00:05',
               '2023-01-01 00:00:06', '2023-01-01 00:00:08',
               '2023-01-01 00:00:09', '2023-01-01 00:00:18',
               '2023-01-01 00:00:47', '2023-01-01 00:00:52',
               '2023-01-01 00:01:07', '2023-01-01 00:01:08',
               ...
               '2024-06-30 23:59:49', '2024-06-30 23:59:49',
               '2024-06-30 23:59:51', '2024-06-30 23:59:51',
               '2024-06-30 23:59:54', '2024-06-30 23:59:54',
               '2024-06-30 23:59:55', '2024-06-30 23:59:56',
               '2024-07-01 00:00:27', '2024-07-01 00:05:23'],
              dtype='datetime64[us]', name='pickup_datetime', length=47089377, freq=None)

In [ ]:
# Crear un nuevo DataFrame con la cantidad de registros por hora
df_hour = df.resample('H').size().to_frame('viajes')

In [10]:
df_hour.tail(2)

,conteo_registros
pickup_datetime,
2024-06-30 23:00:00,1720
2024-07-01 00:00:00,2


In [14]:
df.columns

Index(['dropoff_datetime', 'passenger_count', 'trip_distance', 'PULocationID',
       'DOLocationID', 'total_price', 'service', 'fuel_type',
       'energy_consumption', 'fuel_cost', 'trip_duration', 'labor', 'co2',
       'maintenance', 'net_profit'],
      dtype='object')

### Seleccion de variables

Debido a que nuestra ganancia neta ya es el resultado de restarle al precio total del viaje, sus costos(costo de combustible, mantenimiento, etc) usaremos la variable de ganancia neta,duracion del viaje y cantidad de CO2 emitido.
******
#### <center>Target</center>

El desafio al cual nos ha sometido el cliente, es predecir la demanda de taxis, para lo cual es necesario saber o estimar un valor de cuantos viajes pudieran llegar a realizarse. Por lo cual, se utilizara como target la columna "viajes" creada a partir del recuento de instancias de nuestra data principal transformada en un dataframe con frecuencia horaria.

#### <center>Inputs del modelo</center>

Se consideraron utilizar las variables de ganancia neta, duracion del viaje y cantidad de CO2

In [ ]:
#sum_for_price = df['total_price'].resample('H').sum().to_frame('total_price')
#sum_for_cost = df['fuel_cost'].resample('H').sum().to_frame('fuel_cost')
sum_for_profit = df['net_profit'].resample('H').sum().to_frame('net_profit')
sum_for_duration = df['trip_duration'].resample('H').sum().to_frame('trip_duration')
sum_for_co2 = df['co2'].resample('H').sum().to_frame('co2')

In [ ]:
# Unir los resultados en un solo DataFrame
resultado = df_hour.join([sum_for_profit,sum_for_duration,sum_for_co2])

In [27]:
resultado

,conteo_registros,fuel_cost,net_profit,trip_duration,co2
pickup_datetime,,,,,
2023-01-01 00:00:00,4370,971.372653,97207.925369,974.944444,24258.15776
2023-01-01 01:00:00,4543,1091.092179,102594.328054,989.394167,27604.39664
2023-01-01 02:00:00,3805,955.695558,84971.601142,744.652500,24178.79168
2023-01-01 03:00:00,2853,725.197507,63744.816193,533.422500,18230.76224
2023-01-01 04:00:00,1729,445.140528,37629.893139,307.328333,11231.16080
...,...,...,...,...,...
2024-06-30 20:00:00,2889,568.307699,59838.564601,609.137500,14353.05184
2024-06-30 21:00:00,2967,603.105926,61374.740862,613.465278,15257.32128
2024-06-30 22:00:00,2495,522.075197,50010.844181,485.215556,13230.58096


In [ ]:
df['year'] = df['pickup_datetime'].dt.year
df['quarter'] = df['pickup_datetime'].dt.quarter
df['month'] = df['pickup_datetime'].dt.month
df['day_month'] = df['pickup_datetime'].dt.day
df['weekday'] = df['pickup_datetime'].dt.day_name()
df['hour'] = df['pickup_datetime'].dt.hour
df['weekend'] = df['pickup_datetime'].apply(lambda x: x.weekday() < 5).astype(int) # 1 = weekday, 0 = weekend
#df.drop(columns=['pickup_datetime','dropoff_datetime'],inplace=True)